# Imports

In [1]:
import numpy as np

from data_worker.data_worker import unpickle, unpack_data, combine_batches, split_into_batches
from torch_lib.data_worker import suit4torch
from torch_lib.Interface import Interface
from torch_lib.Nets import LargeNet, MediumNet, SmallNet

# Data

In [2]:
batches_names = [
    'data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4',
    'data_batch_5'
]

data_batches = [
    unpickle(f'datasets/cifar-10-batches-py/{batch_name}') for batch_name
    in batches_names]

unpacked_batches = [
    (unpack_data(data_batch)) for data_batch
    in data_batches]

print(unpacked_batches[0][0].shape)

X, Y = combine_batches(unpacked_batches)

X_torch, Y_torch = suit4torch(X, Y)

batches = split_into_batches(X_torch, Y_torch, 3)

# torch_batches = [(suit4torch(X, Y)) for X, Y in batches]

X_batch0 = batches[0][0]
Y_batch0 = batches[0][1]

print(X.shape, Y.shape)

(10000, 32, 32, 3)
(32, 32, 3)
(50000, 32, 32, 3) (50000,)


# Train network

In [3]:
# net = SmallNet()
# net = MediumNet()
net = LargeNet()

In [4]:
net_interface = Interface(net)

In [ ]:
net_interface.train_net(batches, 1, verbose=False)

# Save weights

In [5]:
# PATH = 'saved_nets/saved_torch/small_v1.pth'
# PATH = 'saved_nets/saved_torch/medium_v1.pth'
PATH = 'saved_nets/saved_torch/large_v1.pth'

In [ ]:
net_interface.save_weights(PATH)

# Predict

In [ ]:
preds = net_interface.predict_net(X_batch0)
preds = preds.detach().numpy()

In [ ]:
print(preds)
print(np.argmax(preds, axis=1), Y_batch0)

# Evaluate Accuracy

In [9]:
acc, N = net_interface.eval_acc_net(X_torch, Y_torch)
print(acc, N)

0.1 50000


# Load weights

In [6]:
net_interface.load_weights(PATH)

In [7]:
preds = net_interface.predict_net(X_torch)
preds = preds.detach().numpy()

c:\users\tronc\appdata\local\programs\python\python39\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [8]:
print(preds)
print(np.argmax(preds, axis=1), Y_torch)

[[-0.03193782 -0.02609414 -0.09416538 ... -0.04211621  0.0363774
  -0.00137505]
 [-0.03180882 -0.02602005 -0.09423827 ... -0.04228754  0.0366769
  -0.00128849]
 [-0.03197443 -0.02589439 -0.09421802 ... -0.04241734  0.03658268
  -0.0012246 ]
 ...
 [-0.03207375 -0.02579184 -0.0941498  ... -0.04242862  0.03647955
  -0.00118262]
 [-0.0313148  -0.02602111 -0.09421885 ... -0.04214914  0.03719144
  -0.00123877]
 [-0.03213761 -0.0258862  -0.09394593 ... -0.04200218  0.03606745
  -0.00131202]]
[8 8 8 ... 8 8 8] tensor([6, 9, 9,  ..., 9, 1, 1])


# Run forward function

In [28]:
from threading import Thread
import time
import psutil
import subprocess
import pandas as pd


class GpuReader(Thread):
    
    def __init__(self):
        self.process = subprocess.Popen(['tegrastats'], stdout=subprocess.PIPE)
        self.stopped = True
        self.values = {}
        super().__init__()
    
    def __enter__(self):
        self.start()
        return self
    
    def __exit__(self, exception_type, exception_value, traceback):
        self.stop()
    
    def start(self):
        self.stopped = False
        super().start()
    
    def stop(self):
        self.stopped = True
    
    def run(self):
        while not self.stopped:
            resp = self.process.stdout.readline().strip().decode('utf-8')
            resp_array = resp.split(' ')
            idx = resp_array.index('GR3D_FREQ')
            self.values['GR3D_FRWQ'] = resp_array[idx + 1]


class CpuGpuTracker(Thread):
    
    def __init__(self, Ts):
        self.Ts = Ts
        self.stopped = True
        self.start_time = None
        self.values = []
        super().__init__()
    
    def __enter__(self):
        self.start()
        return self
    
    def __exit__(self, exception_type, exception_value, traceback):
#         print(exception_type, exception_value, traceback)
        self.stop()
    
    def start(self):
        self.stopped = False
        self.start_time = time.time()
        super().start()
    
    def stop(self):
        self.stopped = True

    def run(self):
        while not self.stopped:
            mem = psutil.virtual_memory()
            cpu_percent = psutil.cpu_percent()
            gpu_percent = 20
            print(mem)
            print(f'{time.time():.8f}')
            self.values.append((time.time() - self.start_time, mem.percent, cpu_percent, gpu_percent))
            time.sleep(self.Ts)
    
    def get_values(self):
        return pd.DataFrame(self.values, columns=['time', 'memory', 'cpu_percent', 'gpu_percent'])        
    

In [42]:
# import sys
# import os
import torch
import time


def execute_net(net_interface, X_data, Y_data, batch_size, priority, loops=1, device='cuda', metrics=['ex_time', 'cpu_max', 'gpu_max', 'mem_max'], echo=True):
    if sys.platform == 'linux':
        os.nice(priority)

    device = torch.device('cpu')
    if device == 'cuda':
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        if echo and not torch.cuda.is_available():
            print("'cuda' device is not available. Using 'cpu' instead.")

    batches = split_into_batches(X_data, Y_data, batch_size)
    
    batch_exec_times = []

    procesor_tracked_values = None
    with CpuGpuTracker(0.1) as tracker:
        
        initial_time = time.time()
    
        for loop in range(loops):
            if echo:
                print('loop:', loop)
            batch_count = 0
            for X_batch, Y_batch in batches:
                start_time = time.time()
                Y_pred = net_interface.predict_net(X_batch)
                batch_time = time.time() - start_time
                if echo:
                    print(f'batch_time: {batch_time:.8f}')
                batch_exec_times.append((loop, batch_count, batch_time, time.time() - initial_time))
                batch_count += 1
        
        procesor_tracked_values = tracker.get_values()
    
    batch_exec_times = pd.DataFrame(batch_exec_times, columns=['loop', 'batch_count', 'batch_time', 'time'])
    return procesor_tracked_values, batch_exec_times

tracked_values, batch_exec_times = execute_net(net_interface, X_torch, Y_torch, 10, 0)


svmem(total=8390193152, available=2630250496, percent=68.7, used=5759942656, free=2630250496)loop:
 0
1634658157.98667622
batch_time: 0.03308678
batch_time: 0.00000000
batch_time: 0.04686213
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.01562119
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
svmem(total=8390193152, available=2624987136, percent=68.7, used=5765206016, free=2624987136)batch_time: 0.01562214
1634658158.09786844

batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.01562095
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.01562166
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch

batch_time: 0.00298214
batch_time: 0.00398779
batch_time: 0.00398731
batch_time: 0.00199080
batch_time: 0.00299454
batch_time: 0.00797486
batch_time: 0.00199437
batch_time: 0.00299191
batch_time: 0.00199413
batch_time: 0.00299311
batch_time: 0.00398993
batch_time: 0.00498700
batch_time: 0.00599146
batch_time: 0.00598288
batch_time: 0.00398827
svmem(total=8390193152, available=2612588544, percent=68.9, used=5777604608, free=2612588544)
1634658159.19620538
batch_time: 0.01795292
batch_time: 0.00997257
batch_time: 0.00398946
batch_time: 0.00498605
batch_time: 0.00299120
batch_time: 0.00299454
batch_time: 0.00199151
batch_time: 0.00199533
batch_time: 0.00199413
batch_time: 0.00499368
batch_time: 0.00198698
batch_time: 0.00299335
batch_time: 0.00299120
batch_time: 0.00199437
batch_time: 0.00300431
batch_time: 0.00198150
batch_time: 0.00199556
batch_time: 0.00199437
batch_time: 0.00199556
batch_time: 0.00199294
batch_time: 0.00200057
batch_time: 0.00236869
batch_time: 0.00000000
batch_time: 

batch_time: 0.00498724
batch_time: 0.00199413
batch_time: 0.00199676
batch_time: 0.00199580
batch_time: 0.00299001
batch_time: 0.00199413
batch_time: 0.00199628
batch_time: 0.00299811
batch_time: 0.00198936
batch_time: 0.00400734
batch_time: 0.00298977
batch_time: 0.00299239
batch_time: 0.00199533
batch_time: 0.00299406
svmem(total=8390193152, available=2597785600, percent=69.0, used=5792407552, free=2597785600)batch_time: 0.00298905
1634658160.28491211

batch_time: 0.00398898
batch_time: 0.00299263
batch_time: 0.00199485
batch_time: 0.00199437
batch_time: 0.00199819
batch_time: 0.00199127
batch_time: 0.00299358
batch_time: 0.00299072
batch_time: 0.00299382
batch_time: 0.00199342
batch_time: 0.00199509
batch_time: 0.00399041
batch_time: 0.00298333
batch_time: 0.00199628
batch_time: 0.00099587
batch_time: 0.00199795
batch_time: 0.00199151
batch_time: 0.00199485
batch_time: 0.00199533
batch_time: 0.00299168
batch_time: 0.00199604
batch_time: 0.00299239
batch_time: 0.00299287
batch_time: 

batch_time: 0.01562119
batch_time: 0.00546408
batch_time: 0.00297761
batch_time: 0.00199628
batch_time: 0.00199556
batch_time: 0.00398684
batch_time: 0.00398970
batch_time: 0.00299191
batch_time: 0.00199461
batch_time: 0.00199509
batch_time: 0.00199533
batch_time: 0.00199652
batch_time: 0.00299215
batch_time: 0.00299573
batch_time: 0.00298810
batch_time: 0.00199366
batch_time: 0.00299215
batch_time: 0.00199819
batch_time: 0.00298905
batch_time: 0.00299168
batch_time: 0.00199604
batch_time: 0.00199151
batch_time: 0.00199461
batch_time: 0.00199461
batch_time: 0.00598502
batch_time: 0.00299883
svmem(total=8390193152, available=2614861824, percent=68.8, used=5775331328, free=2614861824)
1634658161.26349211
batch_time: 0.00199461
batch_time: 0.00199652
batch_time: 0.00298882
batch_time: 0.00194979
batch_time: 0.00299263
batch_time: 0.00199461
batch_time: 0.00299311
batch_time: 0.00199437
batch_time: 0.00698137
batch_time: 0.00299120
batch_time: 0.00099826
batch_time: 0.00199628
batch_time: 

batch_time: 0.00199699
batch_time: 0.00299025
batch_time: 0.00199533
batch_time: 0.00199485
batch_time: 0.00199509
batch_time: 0.00199461
batch_time: 0.00199652
batch_time: 0.00199509
batch_time: 0.00399160
batch_time: 0.00199080
batch_time: 0.00099826
svmem(total=8390193152, available=2596302848, percent=69.1, used=5793890304, free=2596302848)
1634658162.14313769
batch_time: 0.00299096
batch_time: 0.00199461
batch_time: 0.00299191
batch_time: 0.00199389
batch_time: 0.00299287
batch_time: 0.00199366
batch_time: 0.00199509
batch_time: 0.00199413
batch_time: 0.00199509
batch_time: 0.00099635
batch_time: 0.00199580
batch_time: 0.00099707
batch_time: 0.00100517
batch_time: 0.00099754
batch_time: 0.00199318
batch_time: 0.00199413
batch_time: 0.00199509
batch_time: 0.00199413
batch_time: 0.00199461
batch_time: 0.00199485
batch_time: 0.00087333
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 

KeyboardInterrupt: 

In [40]:
tracked_values

,time,memory,cpu_percent,gpu_percent
0,0.003958,69.4,18.1,20
1,0.119735,69.4,69.0,20
2,0.229311,69.4,67.9,20
3,0.335375,69.4,57.1,20
4,0.439186,69.4,60.7,20
...,...,...,...,...
92,10.100331,69.5,80.0,20
93,10.211033,69.4,96.4,20
94,10.321742,69.4,71.4,20
95,10.431443,69.3,89.3,20


In [41]:
batch_exec_times

,loop,batch_count,batch_time,time
0,0,0,0.003223,0.004220
1,0,1,0.002991,0.008209
2,0,2,0.003015,0.012198
3,0,3,0.000996,0.013194
4,0,4,0.001995,0.015190
...,...,...,...,...
4995,0,4995,0.001995,10.603016
4996,0,4996,0.000998,10.604013
4997,0,4997,0.001996,10.606010
4998,0,4998,0.001993,10.608003
